In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data_test=pd.read_csv('../../data/test_airbnb_berlin.xls')
data_train=pd.read_csv('../../data/train_airbnb_berlin.xls')

In [5]:
"""
data viz
"""

data_train.head(15)

# computing number of rows
rows = len(data_train.axes[0])
print('number row :',rows)
# computing number of columns
cols = len(data_train.axes[1])
print('number columns :',cols)

number row : 15692
number columns : 39


In [6]:
data_train.columns.values

array(['Listing ID', 'Listing Name', 'Host ID', 'Host Name', 'Host Since',
       'Host Response Time', 'Host Response Rate', 'Is Superhost',
       'neighbourhood', 'Neighborhood Group', 'City', 'Postal Code',
       'Country Code', 'Country', 'Latitude', 'Longitude',
       'Is Exact Location', 'Property Type', 'Room Type', 'Accomodates',
       'Bathrooms', 'Bedrooms', 'Beds', 'Square Feet', 'Guests Included',
       'Min Nights', 'Reviews', 'First Review', 'Last Review',
       'Overall Rating', 'Accuracy Rating', 'Cleanliness Rating',
       'Checkin Rating', 'Communication Rating', 'Location Rating',
       'Value Rating', 'Instant Bookable', 'Business Travel Ready',
       'Price'], dtype=object)

In [7]:
data_train.head(1)

,Listing ID,Listing Name,Host ID,Host Name,Host Since,Host Response Time,Host Response Rate,Is Superhost,neighbourhood,Neighborhood Group,...,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Business Travel Ready,Price
0,19665213.0,*,156079597.0,Maximilian,2016-01-20,NaN,NaN,f,Prenzlauer Berg,Pankow,...,100.0,10.0,10.0,10.0,10.0,9.0,10.0,t,f,26.0


In [101]:
data_train=data_train.drop(['Listing Name', 'Host ID', 'Host Name', 'Host Since','Is Exact Location',
        'Country', 'First Review', 'Last Review','City', 'Postal Code','Country Code'], axis=1)

In [45]:
#liste des features autour de 18% de "missing"
listfeatures=[] 

def missing(df):
    total = 0
    for col in df.columns:
        miss = df[col].isnull().sum()
        pct = df[col].isna().mean() * 100
        total += miss
        if miss != 0:
            print('{} => {} [{}%]'.format(col, df[col].isnull().sum(), round(pct, 2)))
            if pct>15 and pct <25 :
                listfeatures.append(col)
        #else:
        #    print('{} => {} '.format(col, 'no missing value'))

    if total == 0:
        print("no missing values")
        
missing(data_train)

Listing ID => 1 [0.01%]
Listing Name => 54 [0.34%]
Host Name => 22 [0.14%]
Host Since => 21 [0.13%]
Is Superhost => 23 [0.15%]
City => 1 [0.01%]
Postal Code => 229 [1.46%]
First Review => 2692 [17.19%]
Last Review => 2693 [17.2%]
Overall Rating => 2949 [18.84%]
Accuracy Rating => 2958 [18.89%]
Cleanliness Rating => 2957 [18.89%]
Checkin Rating => 2960 [18.91%]
Communication Rating => 2957 [18.89%]
Location Rating => 2958 [18.89%]
Value Rating => 2959 [18.9%]
True


In [11]:
data_train=data_train.drop(['Square Feet','Host Response Time','Host Response Rate'], axis=1)

KeyError: "['Square Feet', 'Host Response Time', 'Host Response Rate'] not found in axis"

In [14]:
for j in ['Price','Bathrooms','Bedrooms','Beds']:
    for i in data_train[data_train[j].isna()== True].index:
        data_train=data_train.drop(i, axis=0) #drop data where price is NaN

In [25]:
data_train['Checkin Rating'].head(1)

0    10.0
Name: Checkin Rating, dtype: float64

__Recherche de corrélations entre les features présentant un taux de missing value "NaN" autour de 18%__

On va s'aider de la liste **listfeatures** définie plus tôt


In [46]:
#tableau de correlation de "missing" entre deux features
tablefeatures=np.ones((len(listfeatures),len(listfeatures)))

#calcul des corrélations de missing entre deux features
index1=0 
for q in listfeatures:
    nanrows=len(data_train[data_train[q].isna()== True].axes[0])
    index2=0
    for p in listfeatures :
        if q!=p:
            bothmiss=len(data_train[data_train[q].isna()== True][data_train[p].isna()== True].axes[0])
        #print ("P("+q+" et "+p+" vaut NaN) =" + str(bothmiss/nanrows))
            tablefeatures[index1,index2]=bothmiss/nanrows
        index2+=1
    index1+=1   

#affichage du tableau de corrélations
print("Pourcentage minimal de corrélation entre deux features de la liste : " + str(round(tablefeatures.min()*100,2)) + "% \n")
print(tablefeatures)

C:\Users\USER\AppData\Local\Temp\ipykernel_4416\61146275.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bothmiss=len(data_train[data_train[q].isna()== True][data_train[p].isna()== True].axes[0])


Pourcentage minimal de corrélation entre deux features de la liste : 90.95% 

[[1.         1.         1.         1.         1.         1.
  1.         1.         1.        ]
 [0.99962867 1.         1.         1.         1.         1.
  1.         1.         1.        ]
 [0.91285181 0.91319091 1.         1.         1.         1.
  1.         1.         1.        ]
 [0.91007437 0.91041244 0.9969574  1.         0.99966193 1.
  0.99966193 0.99966193 0.99966193]
 [0.91038214 0.91072032 0.99729456 1.         1.         1.
  1.         1.         1.        ]
 [0.90945946 0.9097973  0.99628378 0.99932432 0.99898649 1.
  0.99898649 0.99932432 0.99932432]
 [0.91038214 0.91072032 0.99729456 1.         1.         1.
  1.         1.         1.        ]
 [0.91007437 0.91041244 0.9969574  0.99966193 0.99966193 1.
  0.99966193 1.         1.        ]
 [0.90976681 0.91010477 0.99662048 0.9993241  0.9993241  0.99966205
  0.9993241  0.99966205 1.        ]]


On peut donc raisonnablement affirmer que ces features sont de type MNAR : une listwise deletion n'est donc pas conseillée (et avec 18% d'instances concernées, ce n'est pas idéal)